# Importation des libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors




import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots
import folium 
from folium import plugins
from tqdm.notebook import tqdm as tqdm




import warnings
warnings.filterwarnings('ignore')


import seaborn as sns
import pandas as pd 
import random
import math
from scipy.stats import expon
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from pandas import read_csv
from pandas.plotting import autocorrelation_plot
import scipy.optimize as optim

#from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
plt.style.use('seaborn')
%matplotlib inline 

# Lecture et Preparation des donnees

Nous proposons une modelisation des cas confirmes par jour et des cas confirmes cumulatif

In [2]:
df = pd.read_csv("dailycasenumber_oa.csv",sep=",")

## Preparation des donnees

In [3]:
# cases 
cases = ['Confirmed_cases', 'Dead', 'Recovered', 'Active','Daily_case']

# Active Case = confirmed - deaths - recovered
df['Active'] = df['Confirmed_cases'] - df['Dead'] - df['Recovered']

# filling missing values 
df[['Country']] = df[['Country']].fillna('')
df[cases] = df[cases].fillna(0)

df.head()


,Country,Lat,Long,Date,Confirmed_cases,Dead,Recovered,Daily_case,Active
0,Burkina Faso,12.2383,-1.5616,2020-01-22,0,0,0,0,0
1,Burkina Faso,12.2383,-1.5616,2020-01-23,0,0,0,0,0
2,Burkina Faso,12.2383,-1.5616,2020-01-24,0,0,0,0,0
3,Burkina Faso,12.2383,-1.5616,2020-01-25,0,0,0,0,0
4,Burkina Faso,12.2383,-1.5616,2020-01-26,0,0,0,0,0


In [5]:
# Creating a dataframe with total no of cases for every country
confirmiedcases = pd.DataFrame(df.groupby('Country')['Confirmed_cases'].sum())
confirmiedcases['Country'] = confirmiedcases.index
#confirmiedcases.index = np.arange(1,181)

Deathcases = pd.DataFrame(df.groupby('Country')['Dead'].sum())
Deathcases['Country'] = Deathcases.index
#Deathcases.iodex = np.arange(1,181)

Recoveredcases = pd.DataFrame(df.groupby('Country')['Recovered'].sum())
Recoveredcases['Country'] = Recoveredcases.index
#Recoveredcases.iodex = np.arange(1,181)

Activecases = pd.DataFrame(df.groupby('Country')['Active'].sum())
Activecases['Country'] = Activecases.index
#Activecases.iodex = np.arange(1,181)

global_Activecases = Activecases[['Country','Active']]
global_Deathcases = Deathcases[['Country','Dead']]
global_Recoveredcases = Recoveredcases[['Country','Recovered']]
global_confirmiedcases = confirmiedcases[['Country','Confirmed_cases']]



## Graphique des cas confirmes

In [7]:
fig = px.bar(global_confirmiedcases.sort_values('Confirmed_cases',ascending=False)[:20][::-1],x='Confirmed_cases',y='Country',title='Confirmed Cases West Africa',text='Confirmed_cases', height=900, orientation='h')
fig.show()



## Graphique des cas de deces

In [8]:
fig = px.bar(global_Deathcases.sort_values('Dead',ascending=False)[:20][::-1],x='Dead',y='Country',title='Deaths Cases West Africa',text='Dead', height=900, orientation='h')
fig.show()



## Graphique des cas gueris

In [9]:
fig = px.bar(global_Recoveredcases.sort_values('Recovered',ascending=False)[:20][::-1],x='Recovered',y='Country',title='Recovered Cases West Africa',text='Recovered', height=900, orientation='h')
fig.show()



## Graphique des cas actives

In [10]:
fig = px.bar(global_Activecases.sort_values('Active',ascending=False)[:20][::-1],x='Active',y='Country',title='Active Cases West Africa',text='Active', height=900, orientation='h')
fig.show()

## visualisation globales du Covid en Afrique de l'Ouest

In [11]:
date_c = df.groupby('Date')['Confirmed_cases','Dead','Recovered','Active'].sum().reset_index()


from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=4, subplot_titles=("Comfirmed", "Deaths", "Recovered",'Active'))

trace1 = go.Scatter(
                x=date_c['Date'],
                y=date_c['Confirmed_cases'],
                name="Confirmed",
                line_color='orange',
                mode='lines+markers',
                opacity=0.8)
trace2 = go.Scatter(
                x=date_c['Date'],
                y=date_c['Dead'],
                name="Deaths",
                line_color='red',
                mode='lines+markers',
                opacity=0.8)

trace3 = go.Scatter(
                x=date_c['Date'],
                y=date_c['Recovered'],
                name="Recovered",
                mode='lines+markers',
                line_color='green',
                opacity=0.8)

trace4 = go.Scatter(
                x=date_c['Date'],
                y=date_c['Active'],
                name="Active",
                line_color='blue',
                mode='lines+markers',
                opacity=0.8)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 1, 4)
fig.update_layout(template="plotly_dark",title_text = '<b>Global West Africa  Spread of the Coronavirus Over Time </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='white'))
fig.show()

### Cas particulier du Senegal

In [12]:
grouped_senegal = df[df['Country'] == "Senegal"].reset_index()
grouped_senegal_date = grouped_senegal.groupby('Date')['Date', 'Confirmed_cases', 'Dead','Recovered','Active'].sum().reset_index()


In [13]:
fig = make_subplots(rows=1, cols=4, subplot_titles=("Comfirmed_cases", "Dead", "Recovered",'Active'))

trace1 = go.Scatter(x=grouped_senegal_date['Date'],y=grouped_senegal_date['Confirmed_cases'],name="Confirmed_cases",line_color='orange',mode='lines+markers',opacity=0.8)
trace2 = go.Scatter(x=grouped_senegal_date['Date'],y=grouped_senegal_date['Dead'],name="Dead",line_color='red',mode='lines+markers',opacity=0.8)
trace3 = go.Scatter(x=grouped_senegal_date['Date'],y=grouped_senegal_date['Recovered'],name="Recovered",mode='lines+markers',line_color='green',opacity=0.8)
trace4 = go.Scatter(x=grouped_senegal_date['Date'],y=grouped_senegal_date['Active'],name="Active",line_color='blue',mode='lines+markers',opacity=0.8)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 1, 4)
fig.update_layout(template="plotly_dark",title_text = '<b>Spread of the Coronavirus Over Time in SENEGAL</b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='white'))
fig.show()


# create map and display it
#latitude = 39.91666667
#longitude = 116.383333
#world_map = folium.Map(location=[latitude, longitude], zoom_start=3.5,tiles='Stamen Toner')

#for lat, lon, Confirmed,Deaths,Recovered,name in zip(grouped_senegal['Lat'], grouped_senegal['Long'], grouped_senegal['Confirmed_cases'],grouped_senegal['Dead'],grouped_senegal['Recovered'], grouped_senegal['Country']):
 #   folium.CircleMarker([lat, lon],
  #                      radius=10,
   #                     popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
    #                            '<strong>Confirmed Cases</strong>: ' + str(Confirmed) + '<br>'
     #                           '<strong>Recovered Cases</strong>: ' + str(Recovered) +'<br>'
      #                          '<strong>Deaths Cases</strong>: ' + str(Deaths) +'<br>'),
       #                 color='red',
                        
        #                fill_color='red',
         #               fill_opacity=0.7 ).add_to(world_map)
#world_map

### Cas particulier du  Burkina Faso

In [14]:
grouped_burkina = df[df['Country'] == "Burkina Faso"].reset_index()
grouped_burkina_date = grouped_burkina.groupby('Date')['Date', 'Confirmed_cases', 'Dead','Recovered','Active'].sum().reset_index()



In [15]:
fig = make_subplots(rows=1, cols=4, subplot_titles=("Confirmed", "Deaths", "Recovered",'Active'))

trace1 = go.Scatter(x=grouped_burkina_date['Date'],y=grouped_burkina_date['Confirmed_cases'],name="Confirmed_cases",line_color='orange',mode='lines+markers',opacity=0.8)
trace2 = go.Scatter(x=grouped_burkina_date['Date'],y=grouped_burkina_date['Dead'],name="Dead",line_color='red',mode='lines+markers',opacity=0.8)
trace3 = go.Scatter(x=grouped_burkina_date['Date'],y=grouped_burkina_date['Recovered'],name="Recovered",mode='lines+markers',line_color='green',opacity=0.8)
trace4 = go.Scatter(x=grouped_burkina_date['Date'],y=grouped_burkina_date['Active'],name="Active",line_color='blue',mode='lines+markers',opacity=0.8)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 1, 4)
fig.update_layout(template="plotly_dark",title_text = '<b>Global Spread of the Coronavirus Over Time in BURKINA FASO</b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='white'))
fig.show()

## Mapping des cas confirmes ouest africa

In [16]:
# create map and display it
world_map = folium.Map(location=[10, -20], zoom_start=2.5,tiles='Stamen Toner')

for lat, lon, Confirmed,Deaths,Recovered,name in zip(df['Lat'], df['Long'], df['Confirmed_cases'],df['Dead'],df['Recovered'], df['Country']):
    folium.CircleMarker([lat, lon],
                        radius=7,
                        popup = ('<strong>Country</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(Confirmed) + '<br>'
                                '<strong>Recovered Cases</strong>: ' + str(Recovered) +'<br>'
                                '<strong>Deaths Cases</strong>: ' + str(Deaths) +'<br>'),
                        color='red',
                        
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(world_map)
world_map

In [18]:
formated_gdf = df.groupby(['Date', 'Country'])['Confirmed_cases', 'Dead'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')
formated_gdf['size'] = formated_gdf['Confirmed_cases'].pow(0.3)

fig = px.scatter_geo(formated_gdf, locations="Country", locationmode='country names', 
                     color="Confirmed_cases", size='size', hover_name="Country", 
                     range_color= [0, 1500], 
                     projection="natural earth", animation_frame="Date", 
                     title='COVID-19: Spread Over Time', color_continuous_scale="portland")
# fig.update(layout_coloraxis_showscale=False)
fig.show()